# Подготовка данных

In [1]:
from sklearn import ensemble , cross_validation, metrics, preprocessing,tree,neural_network,linear_model,neighbors,model_selection
#самая распространенная библиотека по машин лернингу
import numpy as np
#линейная алгебра
import pandas as pd
#работа с датасетами
import xgboost as xgb
#градиентный бустинг

/home/stas/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
bank=pd.read_csv("/home/stas/data/bank.csv",sep=";")
#имортируем датасет

In [3]:
bank_vords=bank.iloc[:,[1,2,3,4,6,7,8,10,15,16]]
bank_numeric=bank.iloc[:,[0,5,9,11,12,13,14]]
#делим на вещественный(численные) и строчные признаки

In [4]:
le = preprocessing.LabelEncoder()

X=[]

for i in range(0,10):
    X.append(le.fit_transform(bank_vords.iloc[:,[i]]))
    #преобразовываем строковые к численным и к нампаевскому массиву

X1=np.ones((7,4521))
for i in range(0,7):
    X1[i]=np.array(bank_numeric.iloc[:,[i]]).T
    #тут просто к нампаевскому

/home/stas/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [5]:
y=np.array(X[9])
X_num=np.array(X[:9])
X_col=X1
new_X=np.hstack((X_num.T,X_col.T))
#склеиваем признаки на которых будем обучаться, и выделяем переменную с ответами

# Начинам обучение
проверять качество будем кросс-валидацией с разбитием на 10 фолдов, дабы уменьшить дисперсию

In [108]:
estimator = xgb.XGBClassifier(learning_rate=0.01, max_depth=1000, n_estimators=1000, min_child_weight=3)
cross_validation.cross_val_score(estimator, new_X, y,scoring = 'accuracy', cv = 10).mean()
#Самый универсальный градиентный бустинг, с него и начнем

0.89692756256226913

In [109]:
tree_clf=ensemble.RandomForestClassifier(n_estimators=500,max_depth=4000)
cross_validation.cross_val_score(tree_clf, new_X, y,scoring = 'accuracy', cv = 10).mean()
#Далее мой любимый случайный лес, в 500 деревьев и очень глубокими ветвями

0.89869405536345681

In [111]:
%%time
q=[]
neur_count=[50,10,200,300,500,750,1000,1250,1500]
for i in neur_count:
    neur=neural_network.MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(16,i,2))
    q.append([cross_validation.cross_val_score(neur, new_X, y,scoring = 'accuracy', cv = 10).mean(),i])
    #далее построим многослоиную классифицирующую) перцептронную нейронную сеть первый слой 16,
    #равный колл-ву признаков,2 слой- эксперементально выведем оптимальное колл-во, 3 слой-2 как и ответов
    #lbfgs использовал так-как мне понравилось название это квази Ньютоновский метод
    #пеналь-лассо, альфа 0.00001

CPU times: user 30min 52s, sys: 1min 37s, total: 32min 30s
Wall time: 16min 15s


In [112]:
q
#вроде в среднем точность 88.4%

[[0.88476039774170245, 50],
 [0.88542411455586179, 10],
 [0.88476039774170245, 200],
 [0.84679716345308564, 300],
 [0.88476039774170245, 500],
 [0.88365420305143694, 750],
 [0.88321465549239109, 1000],
 [0.88166305260895916, 1250],
 [0.88453915880364931, 1500]]

In [107]:
log_clf=linear_model.LogisticRegression()
cross_validation.cross_val_score(log_clf, new_X, y,scoring = 'accuracy', cv = 10).mean()
#А теперь старая-добрая логистическая регрессия

0.88852292484713513

In [113]:
poly=preprocessing.PolynomialFeatures(2)
poly_X=poly.fit_transform(new_X)
#Так как точность мне не очень нравится, попробуем увеличить число признаков полиномом,
#и на нем обучить два самых успешных алгоритма
# так как комп у меня не самый сильный, на этот раз будем бить на 5 фолдов

Лучше всего себя показал лес и град.бустинг так что попробуем улучшить качество полиномами

In [114]:
cross_validation.cross_val_score(estimator,poly_X, y,scoring = 'accuracy', cv = 5).mean()

0.89537769520363764

In [115]:
cross_validation.cross_val_score(tree_clf, poly_X, y,scoring = 'accuracy', cv = 5).mean()

0.89559844521586085

И победил лес, почти взяв 90

In [118]:
tree_clf.fit(poly_X,y)
#а вот и наша принимающая решения система, с точностью около 90%

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4000, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

(16,)

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(new_X, y, test_size=0.3)

In [7]:
from keras.layers import Dense, Activation
from keras.models import Sequential
model = Sequential()
model.add(Dense(3184, activation='relu', input_dim=16))
model.add(Dense(64,activation="tanh"))
model.add(Dense(128,activation="linear"))
model.add(Dense(256,activation="sigmoid"))
model.add(Dense(128,activation="tanh"))
model.add(Dense(32,activation="relu"))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

Using TensorFlow backend.


In [11]:
%%time
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
3164/3164 [==============================] - 1s - loss: 0.3662 - acc: 0.8818     

In [13]:
model.evaluate(X_test, y_test,batch_size=1357)

1357/1357 [==============================] - 0s


[0.34339776635169983, 0.89167279005050659]